# GIAN 5: Retrieving Information

You should now be familiar with how natural language processing techniques such as parsing, part-of-speech tagging, lemmatizing, and named-entity recognition can be used in text mining.

In this notebook you will learn basic techniques for identifying informative words in text.

In [ ]:
# load the libraries required for this session
import os
import re
import random
from math import *
from collections import Counter
import spacy
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# and define some elementary functions
def highest(d, n):
    """Give the n highest scoring items in a dictionary"""
    ds=sorted(d.items(), key=lambda x: -x[1])
    return([item for item, value in ds][:n])

def lowest(d, n):
    """Give the n lowest scoring items in a dictionary"""
    ds=sorted(d.items(), key=lambda x: x[1])
    return([item for item, value in ds][:n])

def print_doc_info(doc, searchwords, nsentences=5):
    doc_sentences=[]
    doc=nlp(doc.text)
    print("\n****************")
    print(doc_names[docnum])
    print("\nMOST IMPORTANT NAMED ENTITIES:"),
    print(', '.join(highest(Counter(ent.orth_ for ent in doc.ents),10)))
    print("\nMATCHING SENTENCES:"),
    j=0
    for sent in doc.sents:
        if any([token.lower_ in searchwords for token in sent]):
            print(sent)
            j=j+1
        if j>=nsentences:
            break
    print("****************\n")

def best_matches(searchbase, searchwords, n, match_all = True):
    """Find the n documents where search words are ranked most highly.
    
    Allows for finding any or all search terms with match_all="""
    scores=[]
    for docnum, docwords in searchbase:
        matches=[searchword in docwords for searchword in searchwords]
        if (match_all==True and all(matches)) or (match_all==False and any(matches)):
            match_ranks=[docwords.index(searchword) for i,searchword in enumerate(searchwords) if matches[i]==True]
            scores.append((docnum, sum(match_ranks)))
    return(sorted(scores)[:n])
    
def find_similar(searchbase, docnum, n):
    """Find n documents with the highest overlap in keywords"""
    target_words=set(searchbase[docnum][1])
    results=[]
    for i, docwords in searchbase:
        if docnum==i:
            continue
        matchwords=target_words.intersection(docwords)
        r=len(matchwords)
        results.append((i, r, matchwords))
    results.sort(key=lambda x: x[1], reverse=True)
    return(results[:n])

In [ ]:
# Load the spaCy NLP pipeline
# which resource you load here is dependent on the models you installed
# see https://spacy.io/docs/usage/models
import en_core_web_sm
nlp = en_core_web_sm.load(disable=["parse", "tag", "ner"])

## 1. Using Corpora

In this lesson, we will start working with collections of documents, or, in other words, corpora. 

We will use a sample of the BBC subtitle corpus collected by [Van Heuven, Mandera, Keuleers, & Brysbaert (2014)](http://www.tandfonline.com/doi/abs/10.1080/17470218.2013.850521)

The files have already been cleaned of time stamps, so they are ready to be processed by spaCy.

In [ ]:
corpus_dir="GIAN5_data"
filenames=[filename for filename in os.listdir(corpus_dir) if filename.endswith('srt')]

In [ ]:
my_filenames=random.sample(filenames, 500)

We will now run spaCy's NLP pipeline to tokenize all the files

In [ ]:
docs=[]
doc_names=[]
for filename in my_filenames:
    f=open("{:s}/{:s}".format(corpus_dir,filename), encoding="utf-8")
    s=f.read()
    if len(s)>100:   # process only files with more than 100 characters
        docs.append(nlp(s))
        doc_names.append(filename)

## 2. Document length, vocabulary, and the type to token ratio

When we start out analyzing a corpus, a first thing to learn is the length of the documents in the corpus.

Let us start out by finding out the length of each document and making a histogram of the results. 

In [ ]:
docs_tokens=[[token.lower_ for token in doc if token.is_alpha] for doc in docs]
docs_ntokens=[len(doc) for doc in docs_tokens]

In [ ]:
plt.hist(docs_ntokens)
plt.xlabel("Number of tokens")
plt.ylabel("Number of Documents")
plt.show()

We can see that the histogram shows a concentration of values on the left and a long tail on the right: our corpus is composed of mostly of short documents and very long documents are very rare. 

Let us now move from the number of tokens to the number of types in each document. 

Another way to look at number of types is as the *vocabulary* of the document. The more different words are used, the larger the document's vocabulary. 

As we have learned in previous lectures, we can easily build a frequency dictionary of a document. The number of entries in that frequency dictionary gives us the number of types in a document.

In [ ]:
# First, make a list of frequency dictionaries
docs_fds=[Counter(doc) for doc in docs_tokens]
# Then, get the length of each frequency dictionary
docs_ntypes=[len(fd) for fd in docs_fds]

In [ ]:
# And plot the histogram
plt.hist(docs_ntypes)
plt.xlabel("Number of Types")
plt.ylabel("Number of Documents")
plt.show()

It is likely that longer documents also have more types. But if this were a perfect relation, the histograms above would be identical.

Let's make a scatterplot to gain more insight.

In [ ]:
plt.scatter(docs_ntokens, docs_ntypes)
plt.ylim(0)
plt.xlim(0)
plt.xlabel("Number of tokens")
plt.ylabel("Number of types")
plt.show()

There clearly is a very strong relation! But still there are documents of widely different length that have about the same number of types. This means that in some documents more words are used repeatedly than in other documents. We could say that the former documents have a poorer vocabulary, while documents with more types for the same amount of tokens have a richer vocabulary

$TTR=\frac{ntypes}{ntokens}$

It is easy to see that the ratio between number of types and number of tokens expresses something about the variety of words used in that document: the higher the ratio, the richer the vocabulary.

Let's compute the TTR for each document and plot it against the number of tokens.

In [ ]:
docs_ttrs=[docs_ntypes[i]/docs_ntokens[i] for i in range(len(docs))]

In [ ]:
plt.scatter(docs_ntokens, docs_ttrs)
plt.ylim(0)
plt.xlim(0)
plt.xlabel("Number of tokens")
plt.ylabel("TTR")
plt.show()

We see that, in general, longer documents have a smaller type to token ratio than shorter documents. We also see that the TTR expresses a lot of variation. For documents that are the same length, the TTR adequately expresses the difference in vocabulary.

Looking at the graph, we could now choose some documents that we are interested in comparing.

## 3. Finding informative words with ${TF} \times {IDF}$

As we have seen, frequency dictionaries tell us which words occur frequently in a document. However, we have also seen that the most frequent words in a document are not very informative.

Let's take the top 10 words of the first document in our corpus

In [ ]:
docs_fds[0].most_common(10)

This could be *any* document. However, this insight also gives us a very important clue: If we want to identify words which are important in a document, we need to identify the words which have a high occurence specifically in that document and not in other documents.

So let's first look at words which are not informative by looking at how many documents each word occurs in. If a word is not informative, it will tend to occur in many documents. 

In [ ]:
df=Counter()
for doc_fd in docs_fds:
    df.update(set(doc_fd))

In [ ]:
# Let's look at some common words
df.most_common(10)

In [ ]:
lowest(df, 10)

For the final formula to work we need to transform this document frequency as follows:

For each word divide the total number of documents by the document frequency and take the logarithm of that fraction.

This is also known as the inverse document frequency. It will be lower when a word occurs in more documents.

${IDF}=log(1+\frac{ndocs}{ndocs_{word}})$

In [ ]:
n=len(docs)
idf={word: log(1+n/f) for word, f in df.items()}

In [ ]:
highest(idf,10)

We also need to transform our raw frequencies so that they become log scaled. 

This is called term frequency or $tf$.

${TF}=log(1+f_{word})$

In [ ]:
docs_tfs=[]
for doc_fd in docs_fds:
    doc_tf={word:log(1+f) for word, f in doc_fd.items()}
    docs_tfs.append(doc_tf)

In [ ]:
docs_tfs[0]

Now we can multiply $TF$ and $IDF$ for all the words in all the documents

In [ ]:
docs_tfidfs=[]
for doc_tf in docs_tfs:
    doc_tfidf={word: tf*idf[word] for word, tf in doc_tf.items()}
    docs_tfidfs.append(doc_tfidf)

And now we can find the words with the best ${TF}\times{IDF}$  in each document.

In [ ]:
docnum=99
doc=docs_tfidfs[docnum]
docname=doc_names[docnum]
print(highest(doc, 20))
print(docname)

## 4. Finding documents in which particular words are informative

Now that we have computed ${TF}\times{IDF}$ scores, we can also define words that we are particularly interested in and rank the documents according to the score they give for that word.

Note that using pure term frequency  would give exactly the same result as ${TF}\times{IDF}$ when we have all the word frequencies at our disposal. However, with ${TF}\times{IDF}$, we can simply characterize documents by their most informative words and throw away the information about all the other words. This makes searching much faster.

In [ ]:
# make a search base containing each document's number and its n most informative words
n=200
my_searchbase = [(i, highest(docs_tfidfs[i], 200)) for i in range(len(docs))]

In [ ]:
my_searchbase[120]

In [ ]:
searchwords=["chicken"]
matchlist=best_matches(my_searchbase, searchwords, 5, match_all=True)
matchlist

In [ ]:
doc_names[128]

(You can change parameters in the cell above: use more searchwords and see what the effect is of setting match_all to False)

We have found documents but we still don't know why the documents are interesting. 

We can start by looking at the other words with a high ${TF}\times{IDF}$ in the documents we found.

In [ ]:
for docnum, rank in matchlist:
    print(my_searchbase[docnum][1][:10])

And with a little more effort we can find out more about the document that matches our search.

Here, we output the top named entities and the sentences that the searchword occurs in.

In [ ]:
for docnum, rank in matchlist:
    print_doc_info(docs[docnum], searchwords)

## 5. Finding similar documents

An interesting application of corpora is finding similar documents. There are many different ways to do this.

Here, we illustrate a particularly simple method.

Given our search base in which each document is listed with its top 50 keywords, we can focus on one document and find out which other documents share most of the same keywords.

In [ ]:
my_docnum=matchlist[1][0]
similar_documents=find_similar(my_searchbase, my_docnum, 10)

In [ ]:
for docnum, rank, docwords in similar_documents:
    print("DOCUMENT:", docnum, "; MATCHING KEYWORDS:", ', '.join(docwords))
    print_doc_info(docs[docnum], searchwords)